In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.metrics.pairwise import linear_kernel

https://heartbeat.fritz.ai/recommender-systems-with-python-part-i-content-based-filtering-5df4940bd831

In [2]:
import os
os.getcwd()

'/Users/yangxiang/A List/Uchicago/2020_Spring/Machine_Learning/Project/Submission/Model'

In [5]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
ds = pd.read_csv('/Users/yangxiang/A List/Uchicago/2020_Spring/Machine_Learning/Project/Cocat_review.csv')


In [6]:
#drop '_id'
ds=ds.drop(['_id'], axis=1)

In [7]:
#Change _id to numeric number
ds['_id'] = ds.index

## Creating a TF-IDF Vectorizer
The TF*IDF algorithm is used to weigh a keyword in any document and assign the importance to that keyword based on the number of times it appears in the document. Put simply, the higher the TF*IDF score (weight), the rarer and more important the term, and vice versa.
* Each word or term has its respective TF and IDF score. 
* The TF (term frequency) of a word is the number of times it appears in a document. 
* The IDF (inverse document frequency) of a word is the measure of how significant that term is in the whole corpus.

In [8]:
tf = TfidfVectorizer(analyzer='word', ngram_range=(1, 3), min_df=0, stop_words='english')
tfidf_matrix = tf.fit_transform(ds['as_review'])

the tfidf_matrix is the matrix containing each word and its TF-IDF score with regard to each document


## calculate the relevance or similarity of one document to another.

The method of calculating the user’s likes / dislikes / measures is calculated by taking the cosine of the angle between the user profile vector (Ui ) and the document vector; or in our case, the angle between two document vectors.
The ultimate reason behind using cosine is that the value of cosine will increase as the angle between vectors with decreases, which signifies more similarity.

In [9]:
cosine_similarities = linear_kernel(tfidf_matrix, tfidf_matrix)


In [10]:
results = {}

for idx, row in ds.iterrows():
    similar_indices = cosine_similarities[idx].argsort()[:-100:-1]
    similar_items = [(cosine_similarities[idx][i], ds['_id'][i]) for i in similar_indices]

    results[row['_id']] = similar_items[1:]

we’ve calculated the cosine similarity of each item with every other item in the dataset, and then arranged them according to their similarity with item i, and stored the values in results.

## Making a recommendation


In [11]:
def item(id):
    return ds.loc[ds['_id'] == id]['name'].tolist()[0].split(' - ')[0]

In [30]:
def recommend_tfidf(item_id, num):
    print("Recommending " + str(num) + " products similar to " + item(item_id) + "...")
    print("-------")
    recs = results[item_id][:num]
    for rec in recs:
        print("Recommended: " + item(rec[1]) + " (score:" + str(rec[0]) + ")")

In [31]:
recommend_tfidf(item_id=17, num=5)

Recommending 5 products similar to Banzai Sushi...
-------
Recommended: Akaihana Sushi & Grill (score:0.0261583874614779)
Recommended: Zenshin Asian Restaurant (score:0.02425661625517246)
Recommended: Sushi Ave (score:0.02418907353814648)
Recommended: Sushi San (score:0.02363079636706018)
Recommended: KUMI by Chef Akira Back (score:0.023542859836310034)


## Countervectorizor

In [24]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(analyzer='word', ngram_range=(1, 3), min_df=0, stop_words='english')
cv_matrix = cv.fit_transform(ds['as_review'])

In [25]:
cosine_similarities_cv = linear_kernel(cv_matrix, cv_matrix)

In [26]:
results_cv = {}

for idx, row in ds.iterrows():
    similar_indices_cv = cosine_similarities_cv[idx].argsort()[:-100:-1]
    similar_items_cv = [(cosine_similarities_cv[idx][i], ds['_id'][i]) for i in similar_indices_cv]

    results_cv[row['_id']] = similar_items_cv[1:]

In [27]:
def item_cv(id):
    return ds.loc[ds['_id'] == id]['name'].tolist()[0].split(' - ')[0]

In [28]:
def recommend_cv(item_id, num):
    print("Recommending " + str(num) + " products similar to " + item_cv(item_id) + "...")
    print("-------")
    recs = results_cv[item_id][:num]
    for rec in recs:
        print("Recommended: " + item_cv(rec[1]) + " (score:" + str(rec[0]) + ")")

# Compare Models

In [42]:
recommend_cv(item_id=17, num=5)

Recommending 5 products similar to Banzai Sushi...
-------
Recommended: Flamingo Las Vegas Hotel & Casino (score:7541.0)
Recommended: Mesa Grill (score:7038.0)
Recommended: Gangnam Asian BBQ Dining (score:4027.0)
Recommended: Trump International Hotel Las Vegas (score:3987.0)
Recommended: Joe's Farm Grill (score:3964.0)


In [43]:
recommend_tfidf(item_id=17, num=5)

Recommending 5 products similar to Banzai Sushi...
-------
Recommended: Akaihana Sushi & Grill (score:0.0261583874614779)
Recommended: Zenshin Asian Restaurant (score:0.02425661625517246)
Recommended: Sushi Ave (score:0.02418907353814648)
Recommended: Sushi San (score:0.02363079636706018)
Recommended: KUMI by Chef Akira Back (score:0.023542859836310034)


In [41]:
# Review Example
ds.as_review[1]

'[{\'text\': \'UMGS does an amazing job serving the community. They perform quality work and treat all of their customers like family. Highly recommend\'}, {\'text\': \'I called Connie needing some stuff done on a Saturday. Even though they do not usually work on Saturdays, she was able to send Jacob out on Saturday. Jacob was great! Installed a ceiling fan, window film, a shelf, new thermostat, padlock on the garage and adjusted a wobbly sink faucet. Our Husky decided to escape when Jacob opened the door and Jacob ran after him and caught him and carried all 65 pounds of him back home. I especially like that they are a family business. I work for an insurance agency and will recommend UMGS for sure!\'}, {\'text\': \'We are selling our home and needed our back facia touched up, sanded and painted. Use My Guy Services came out on the appointment day and on time and got the facia looking brand new. They were able to match the paint and get the job done. I would and do recommend them to e

In [45]:
ds.shape

(15095, 13)